In [103]:
%pip install rapidfuzz

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.3 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load Dataset

In [53]:
import pandas as pd
import sqlite3

df = pd.read_csv('./FINAL FOOD DATASET/FOOD-DATA-GROUP1.csv')

print("Preview of your data:")
print(df.head())
print(f"\nShape: {df.shape[0]} rows, {df.shape[1]} columns")

Preview of your data:
   Unnamed: 0.1  Unnamed: 0                              food  Caloric Value  \
0             0           0                      cream cheese             51   
1             1           1                 neufchatel cheese            215   
2             2           2  requeijao cremoso light catupiry             49   
3             3           3                    ricotta cheese             30   
4             4           4              cream cheese low fat             30   

    Fat  Saturated Fats  Monounsaturated Fats  Polyunsaturated Fats  \
0   5.0             2.9                   1.3                 0.200   
1  19.4            10.9                   4.9                 0.800   
2   3.6             2.3                   0.9                 0.000   
3   2.0             1.3                   0.5                 0.002   
4   2.3             1.4                   0.6                 0.042   

   Carbohydrates  Sugars  ...  Calcium  Copper   Iron  Magnesium  Mang

## Create Database

In [5]:
conn = sqlite3.connect('Nutrition.db')
df.to_sql('nutrition_facts', conn, if_exists='replace', index=False)
conn.close()

## Query DB

In [7]:
conn = sqlite3.connect('Nutrition.db')

query = """
SELECT food, `Caloric Value`, Fat, Protein, Carbohydrates
FROM nutrition_facts
WHERE `Caloric Value` < 100
ORDER BY `Caloric Value` DESC
LIMIT 10
"""

result = pd.read_sql_query(query, conn)
print(result)

conn.close()

                       food  Caloric Value  Fat  Protein  Carbohydrates
0            shark fin soup             99  4.3      6.9            8.2
1              swiss cheese             98  7.7      6.7            0.4
2          limburger cheese             98  8.2      6.0            0.1
3               tomato soup             98  0.7      2.2           22.5
4  scrambled eggs mcdonalds             98  7.5      7.7            0.9
5         queen crab cooked             98  1.3     20.2            0.0
6  english muffin mcdonalds             97  2.3      5.7            2.9
7                 crab soup             95  0.8     10.4           12.5
8      chunky peanut butter             94  8.0      3.8            3.5
9                    omelet             94  7.1      6.4            0.4


In [126]:
conn = sqlite3.connect('Nutrition.db')

df2 = pd.read_csv('./FINAL FOOD DATASET/FOOD-DATA-GROUP2.csv')
df3 = pd.read_csv('./FINAL FOOD DATASET/FOOD-DATA-GROUP3.csv')
df4 = pd.read_csv('./FINAL FOOD DATASET/FOOD-DATA-GROUP4.csv')
df5 = pd.read_csv('./FINAL FOOD DATASET/FOOD-DATA-GROUP5.csv')

df2.to_sql('nutrition_facts', conn, if_exists='append', index=False)
df3.to_sql('nutrition_facts', conn, if_exists='append', index=False)
df4.to_sql('nutrition_facts', conn, if_exists='append', index=False)
df5.to_sql('nutrition_facts', conn, if_exists='append', index=False)

conn.close()

In [176]:
conn = sqlite3.connect('Nutrition.db')
query = """
SELECT food, `Caloric Value`, Fat, Protein, Carbohydrates
FROM nutrition_facts
WHERE food = 'groundbeef';
"""

result = pd.read_sql_query(query, conn)
print(result)

conn.close()

Empty DataFrame
Columns: [food, Caloric Value, Fat, Protein, Carbohydrates]
Index: []


In [130]:
conn = sqlite3.connect('Nutrition.db')
query = """
SELECT COUNT(*)
FROM nutrition_facts;
"""

result = pd.read_sql_query(query, conn)
print(result)

conn.close()

   COUNT(*)
0      2395


# Parse User Question for Column Names

In [177]:
# CONSTANTS
user_q = "ground beef"
food_list = [
    "rice vinegar",
    "apple cider vinegar",
    "soy sauce",
    "chicken breast",
    "garlic",
    "onion",
    "olive oil",
    "apple",
    "cider",
    "vinegar",
]
food_set = set(food_list)
food_embeddings = build_food_embeddings(food_list)

## Hashset Exact Match
Finds exact word matches between user query and database column names

In [282]:
import re
from rapidfuzz import fuzz, process

def preprocess(user_q):
    user_q = user_q.lower()
    user_q = re.sub(r"[^a-z0-9\s]", "", user_q)
    tokens = user_q.split()
    return tokens

def fuzzy_match_foods(token, food_set, min_score=80):
    matches = []
    for food in food_set:
        score = fuzz.ratio(token, food)
        if score >= min_score:
            matches.append((food, score))
    return matches

def fuzzy_token_spellcheck(tokens, food_list):  
    for i, tok in enumerate(tokens):
        if len(tok) <= 3:
            continue
        matches = fuzzy_match_foods(tok, food_list)
        if matches:
            # Keep the best match only
            best_food, _ = max(matches, key=lambda x: x[1])
            tokens[i] = best_food
    return tokens

def generate_ngrams(tokens, max_n=3):
    ngrams = []
    n = len(tokens)
    for size in range(max_n, 0, -1):  # start big!
        for i in range(n - size + 1):
            ngram = " ".join(tokens[i:i+size])
            ngrams.append(ngram)
    return ngrams

# This function was generated by Claude [Prompt found in Refrence 1]
def hash_match(query, food_set, max_ngram=4):
    tokens = preprocess(query)
    tokens = fuzzy_token_spellcheck(tokens, food_set)
    found = []
    used_indices = set()  # Track which token positions have been used
    
    # Iterate from largest to smallest n-gram size
    for size in range(max_ngram, 0, -1):
        i = 0
        while i <= len(tokens) - size:
            # Skip if any token in this range has already been used
            if any(idx in used_indices for idx in range(i, i + size)):
                i += 1
                continue
            
            # Generate the n-gram
            ngram = " ".join(tokens[i:i+size])
            # Check if it matches a food
            if ngram in food_set:
                found.append(ngram)
                # Mark these indices as used
                for idx in range(i, i + size):
                    used_indices.add(idx)
                # Move past this match
                i += size
            elif ngram + " raw" in food_set:
                found.append(ngram + " raw")
                # Mark these indices as used
                for idx in range(i, i + size):
                    used_indices.add(idx)
                # Move past this match
                i += size
            else:
                i += 1
    
    return found, used_indices

hash_match(real_user_query, real_food_set)

(['cream cheese', 'chicken breast raw', 'mushrooms raw'], {2, 3, 5, 6, 9})

## Backup Embedding Similarity
Catches misspelled words and other mishaps not caught by exact match

### Embed Food list

In [167]:
from sentence_transformers import SentenceTransformer
import numpy as np

# small, fast, good for semantic food names
model = SentenceTransformer("all-MiniLM-L6-v2")

def build_food_embeddings(food_list):
    food_embeddings = model.encode(food_list, convert_to_numpy=True, normalize_embeddings=True)
    return food_embeddings

food_embeddings = build_food_embeddings(food_list)

In [290]:
def embedding_fallback(query, food_list, food_embeddings, top_k=10, c=.0):
    query_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = np.dot(food_embeddings, query_emb)
    
    top_idx = np.argsort(sims)[::-1][:top_k] 
    candidates = [(food_list[i], float(sims[i])) for i in top_idx]
    
    filtered = [cand for cand in candidates if cand[1] >= c]
    embedding_matches = filtered #[cand for cand in filtered if cand not in hash_matches]
    
    return embedding_matches
embedding_fallback("unprocessed",real_food_list, real_food_embeddings)

[('spelt raw', 0.3248884081840515),
 ('drum raw', 0.3018554449081421),
 ('teaseed oil', 0.2878323793411255),
 ('drum cooked', 0.2823004126548767),
 ('salsify raw', 0.28024783730506897),
 ('prunes', 0.2785925269126892),
 ('prunes stewed', 0.2721095085144043),
 ('prune juice', 0.2720765471458435),
 ('nopales raw', 0.2711069881916046),
 ('pear dried', 0.2695382535457611)]

In [288]:
def embedding_fallback_tokens(query, food_list, food_embeddings, top_k=3, c=.70):
    tokens = preprocess(query)
    embedding_matches = []
    for tok in tokens:
        tok_emb = model.encode([tok], convert_to_numpy=True, normalize_embeddings=True)[0]
        sims = np.dot(food_embeddings, tok_emb)
    
        top_idx = np.argsort(sims)[::-1][:top_k] 
        candidates = [(food_list[i], float(sims[i])) for i in top_idx]

        filtered = [cand for cand in candidates if cand[1] >= c]   
        filtered_strings = [item[0] for item in filtered]
        embedding_matches = embedding_matches + filtered_strings

    embedding_matches = list(dict.fromkeys(embedding_matches))
    return embedding_matches

embedding_fallback_tokens("I have cream cheese, wine, chicken breast, pork, and mushrooms in my fridge",real_food_list, real_food_embeddings)

['cream cheese',
 'goat cheese',
 'cheddar cheese',
 'red wine',
 'white wine',
 'cooking wine',
 'chicken sandwich',
 'lemon chicken',
 'meatless chicken',
 'pork skin',
 'pork bologna',
 'pork loin raw',
 'mushrooms cooked',
 'maitake mushrooms',
 'mushrooms raw']

In [286]:
def embedding_fallback_ngrams(query, food_list, food_embeddings, top_k=1, c=.70):
    tokens = preprocess(query)
    ngrams = generate_ngrams(tokens)
    embedding_matches = []
    for ngram in ngrams:
        ngram_emb = model.encode([ngram], convert_to_numpy=True, normalize_embeddings=True)[0]
        sims = np.dot(food_embeddings, ngram_emb)
    
        top_idx = np.argsort(sims)[::-1][:top_k] 
        candidates = [(food_list[i], float(sims[i])) for i in top_idx]
    
        filtered = [cand for cand in candidates if cand[1] >= c]   
        filtered_strings = [item[0] for item in filtered]
        embedding_matches = embedding_matches + filtered_strings

    embedding_matches = list(dict.fromkeys(embedding_matches))
    return embedding_matches

embedding_fallback_ngrams("I have cream cheese, wine, chicken breast, and mushrooms in my fridge",real_food_list, real_food_embeddings)

have cream cheese
[('cream cheese', 0.8746776580810547)]
cream cheese wine
[('cream cheese', 0.8191268444061279)]
chicken breast and
[('chicken breast raw', 0.7547982931137085)]
breast and mushrooms
[('maitake mushrooms', 0.7185052633285522)]
mushrooms in my
[('mushrooms cooked', 0.7375491857528687)]
cream cheese
[('cream cheese', 1.0000001192092896)]
cheese wine
[('table wine', 0.7596521377563477)]
wine chicken
[('lemon chicken', 0.7232227921485901)]
chicken breast
[('chicken breast raw', 0.7704194784164429)]
mushrooms in
[('mushrooms cooked', 0.7915263175964355)]
cream
[('cream cheese', 0.7722978591918945)]
cheese
[('cream cheese', 0.822540283203125)]
wine
[('red wine', 0.8363298177719116)]
chicken
[('chicken sandwich', 0.7917603254318237)]
mushrooms
[('mushrooms cooked', 0.801211953163147)]


['cream cheese',
 'chicken breast raw',
 'maitake mushrooms',
 'mushrooms cooked',
 'table wine',
 'lemon chicken',
 'red wine',
 'chicken sandwich']

In [259]:
def embedding_sim_test(query, food_list, food_embeddings, top_k=10):
    print(query)
    query_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = np.dot(food_embeddings, query_emb)
    
    top_idx = np.argsort(sims)[::-1][:top_k] 
    candidates = [(food_list[i], float(sims[i])) for i in top_idx]
    return candidates

embedding_sim_test("I have cream cheese, wine, chicken breast, pork, and mushrooms in my fridge",real_food_list, real_food_embeddings)

I have cream cheese, wine, chicken breast, pork, and mushrooms in my fridge


[('cream cheese', 0.5539529919624329),
 ('mushrooms raw', 0.5100688934326172),
 ('cottage cheese creamed', 0.5043703317642212),
 ('cream of mushroom soup', 0.5021468997001648),
 ('mushrooms cooked', 0.5009106993675232),
 ('monterey cheese', 0.4958856701850891),
 ('cream cheese fat free', 0.4870794117450714),
 ('portabella mushrooms raw', 0.4865381419658661),
 ('frijoles with cheese', 0.4795270562171936),
 ('nachos with cheese', 0.47944411635398865)]

# Putting it all together

In [262]:
def match_food(query, food_list, food_set, food_embeddings, max_ngram=4):
    hash_matches = hash_match(query, food_set, max_ngram=max_ngram)
    
    embedding_matches = embedding_fallback_tokens(query, food_list, food_embeddings, hash_matches, top_k=10)
    
    return {"method": "hashset", "matches": hash_matches}, {"method": "embedding", "matches": embedding_matches}

match_food(user_q, food_list, food_set, food_embeddings)

AttributeError: 'NoneType' object has no attribute 'append'

# Querying DB with retrieved food names

In [249]:
conn = sqlite3.connect('Nutrition.db')
query = """
SELECT food
FROM nutrition_facts;
"""
real_food_df = pd.read_sql_query(query, conn)
conn.close()

real_food_list = real_food_df['food'].tolist()

real_food_set = set(real_food_list)
real_food_embeddings = build_food_embeddings(real_food_list)

In [219]:
if 'chicken breast' in real_food_set:
    print(True)

In [232]:
real_user_query = "I have cream cheese, wine, chicken breast, pork, and mushrooms in my fridge"
hashset_dict, embedding_dict = match_food(real_user_query, real_food_list, real_food_set, real_food_embeddings)
match_list = hashset_dict['matches'] # + embedding_dict['matches']
print(match_list)

['cream cheese', 'chicken breast raw', 'mushrooms raw']


In [217]:
conn = sqlite3.connect('Nutrition.db')
placeholders = ','.join('?' * len(match_list))
query = f"""
SELECT *
FROM nutrition_facts
WHERE food IN ({placeholders});
"""

result = pd.read_sql_query(query, conn, params=match_list)
print(result)

conn.close()

   Unnamed: 0.1  Unnamed: 0           food  Caloric Value  Fat  \
0             0           0   cream cheese             51  5.0   
1           708         708  mushrooms raw             15  0.2   

   Saturated Fats  Monounsaturated Fats  Polyunsaturated Fats  Carbohydrates  \
0           2.900                   1.3                   0.2            0.8   
1           0.052                   0.0                   0.1            2.3   

   Sugars  ...  Calcium  Copper   Iron  Magnesium  Manganese  Phosphorus  \
0     0.5  ...    0.008    14.1  0.082      0.027      1.300       0.091   
1     1.4  ...    2.100     0.2  0.400      6.300      0.042      60.200   

   Potassium  Selenium   Zinc  Nutrition Density  
0       15.5    19.100  0.039               7.07  
1      222.6     0.061  0.400               9.40  

[2 rows x 37 columns]


# Reference and AI Prompts

[1]
Claude prompt:
```import re
def preprocess(user_q):
    user_q = user_q.lower()
    user_q = re.sub(r"[^a-z0-9\s]", "", user_q)
    tokens = user_q.split()
    return tokens
def generate_ngrams(tokens, max_n=4):
    ngrams = []
    n = len(tokens)
    for size in range(max_n, 0, -1):  # start big!
        for i in range(n - size + 1):
            ngram = " ".join(tokens[i:i+size])
            ngrams.append(ngram)
    return ngrams
def match_foods_hashset(query, food_set, max_ngram=4):
    tokens = preprocess(query)
    ngrams = generate_ngrams(tokens, max_n=max_ngram)
    found = []
    for ng in reversed(ngrams):
        if ng in food_set:
            found.append(ng)
    return found

user_q= "I have apple cider vinegar and onion in my fridge"
food_list = [
    "rice vinegar",
    "apple cider vinegar",
    "soy sauce",
    "chicken breast",
    "garlic",
    "onion",
    "olive oil",
    "apple",
    "cider",
    "vinegar",
]
```
"I've got this code to find matches in food lists, I put an example of a user query and food list above. The problem right now is the code will return vinegar, cider, apple and apple cider vinegar. I could have it only return the largest ngram, but that will cut out onion. So what I want to do I check for ngram matches between each iteration of ngram generation then if there is a match remove the words. 

Of course we will start with larger n-gram. for example we will generate 4-grams for the user query and find not match, then we will generate 3-grams and find 'apple cider vinegar', we save it and remove 'apple', 'cider', and 'vinegar' from the token list. then we find nothing in 2 ngram and in 1-gram we find only onion since the other words have been removed.

can you write up a script for that?"